In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
import optuna
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [2]:
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if (tn + fp) > 0 else 0

def evaluate_model_on_folds(model, folds=5, prefix="data/fold_", disp=0):
    metrics = {
        'f1': [],
        'acc': [],
        'recall': [],
        'specificity': [],
        'tp': [],
        'fp': [],
        'tn': [],
        'fn': [],
        'conf_matrices': []
    }

    total_conf_matrix = np.zeros((2, 2), dtype=int)
    for i in range(folds):
        train = pd.read_csv(f"{prefix}{i+1}_train.csv")
        test = pd.read_csv(f"{prefix}{i+1}_test.csv")
        
        test['arrytmia'] = test['arrytmia'].apply(lambda x: 1 if x > 1 else x)
        train['arrytmia'] = train['arrytmia'].apply(lambda x: 1 if x > 1 else x)

        scaler = MinMaxScaler()
        train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)

        test = pd.DataFrame(scaler.transform(test), columns=test.columns)

        X_train = train.iloc[: ,1:].values
        y_train = train['arrytmia'].values

        X_test = test.iloc[: ,1:].values
        y_test = test['arrytmia'].values
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        cm = confusion_matrix(y_test, y_pred)
        total_conf_matrix += cm

    # Rozpakowanie łącznej macierzy
    tn, fp, fn, tp = total_conf_matrix.ravel()

    accuracy = (tp + tn) / (tp + tn + fp + fn)

    precision=  tp / (tp+fp)if (tp+fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    F1 = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0

    if disp:
        print(f"F1:           {F1:.4f}")
        print(f"Accuracy:     {accuracy:.4f}")
        print(f"Precision:    {precision:.4f}")
        print(f"Sensitivity:  {sensitivity:.4f}")
        print(f"Specificity:  {specificity:.4f}")
        print(f"TP:           {tp}")
        print(f"FP:           {fp}")
        print(f"TN:           {tn}")
        print(f"FN:           {fn}")
        print("         Pred 1    Pred 0")
        print(f"True 1    {tp:4}     {fn:4}")
        print(f"True 0    {fp:4}     {tn:4}")

    return F1

## KNN

In [3]:
n_trials=20

In [4]:
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 200) # Liczba sąsiadów
    weights = trial.suggest_categorical('weights', ['uniform', 'distance']) #Sposób ważenia sąsiadów
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    metric=trial.suggest_categorical('metric',['euclidean','manhattan'])
    
    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm,
        metric=metric,
        n_jobs=-1
    )


    score=evaluate_model_on_folds(model)
    return round(score,4) 

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials)

best_params = study.best_params
print("Najlepsze parametry:", best_params)

[I 2025-04-13 22:37:24,189] A new study created in memory with name: no-name-2e913b86-9365-4a6f-a266-8899025dd2d5
[I 2025-04-13 22:37:32,987] Trial 0 finished with value: 0.9658 and parameters: {'n_neighbors': 184, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}. Best is trial 0 with value: 0.9658.
[I 2025-04-13 22:37:45,398] Trial 1 finished with value: 0.9728 and parameters: {'n_neighbors': 25, 'weights': 'distance', 'algorithm': 'kd_tree', 'metric': 'manhattan'}. Best is trial 1 with value: 0.9728.
[I 2025-04-13 22:38:00,776] Trial 2 finished with value: 0.9692 and parameters: {'n_neighbors': 78, 'weights': 'distance', 'algorithm': 'kd_tree', 'metric': 'manhattan'}. Best is trial 1 with value: 0.9728.
[I 2025-04-13 22:38:06,224] Trial 3 finished with value: 0.9667 and parameters: {'n_neighbors': 181, 'weights': 'uniform', 'algorithm': 'auto', 'metric': 'euclidean'}. Best is trial 1 with value: 0.9728.
[I 2025-04-13 22:38:15,666] Trial 4 finished with value: 0.972

Najlepsze parametry: {'n_neighbors': 8, 'weights': 'uniform', 'algorithm': 'ball_tree', 'metric': 'manhattan'}


In [5]:
best_model = KNeighborsClassifier(**best_params)
score=evaluate_model_on_folds(best_model,disp=1)

F1:           0.9753
Accuracy:     0.9627
Precision:    0.9656
Sensitivity:  0.9853
Specificity:  0.8960
TP:           47675
FP:           1700
TN:           14640
FN:           712
         Pred 1    Pred 0
True 1    47675      712
True 0    1700     14640


## DecisionTree

In [6]:
def objective(trial):
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy','log_loss']) 
    splitter = trial.suggest_categorical('splitter', ['best', 'random'])
    max_depth = trial.suggest_int('max_depth', 3, 20) 
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    #DODAJ CCP_ALPHA

    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        splitter=splitter,
        random_state=42
    )


    score=evaluate_model_on_folds(model)
    return round(score,4)  

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials)

best_params = study.best_params
print("Najlepsze parametry:", best_params)

[I 2025-04-13 22:43:16,247] A new study created in memory with name: no-name-65ba793c-fa6f-4d3f-967a-272cc1abee2e
[I 2025-04-13 22:43:17,580] Trial 0 finished with value: 0.939 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 4, 'min_samples_split': 13, 'min_samples_leaf': 14, 'max_features': 'log2'}. Best is trial 0 with value: 0.939.
[I 2025-04-13 22:43:18,617] Trial 1 finished with value: 0.943 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 5, 'min_samples_split': 12, 'min_samples_leaf': 19, 'max_features': None}. Best is trial 1 with value: 0.943.
[I 2025-04-13 22:43:25,109] Trial 2 finished with value: 0.9666 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 12, 'min_samples_split': 17, 'min_samples_leaf': 7, 'max_features': None}. Best is trial 2 with value: 0.9666.
[I 2025-04-13 22:43:26,046] Trial 3 finished with value: 0.8862 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 6, '

Najlepsze parametry: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 20, 'min_samples_split': 18, 'min_samples_leaf': 10, 'max_features': None}


In [7]:
best_model = DecisionTreeClassifier(**best_params, random_state=42)
score=evaluate_model_on_folds(best_model,disp=1)

F1:           0.9674
Accuracy:     0.9513
Precision:    0.9674
Sensitivity:  0.9674
Specificity:  0.9034
TP:           46810
FP:           1578
TN:           14762
FN:           1577
         Pred 1    Pred 0
True 1    46810     1577
True 0    1578     14762


## RandomForestClassifier

In [8]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200) # liczba drzew w lesie
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    max_depth = trial.suggest_int('max_depth', 5, 50) # Maksymalna głebokość drzewa
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20) # Minimalna liczba próbek w węźle wymagana do podzielenia
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20) # minimalna liczba próbek w liściu
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2']) # Liczba cech rozważanych podczas rodzielania węzła
    
    model = RandomForestClassifier(
        criterion=criterion,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    
    score=evaluate_model_on_folds(model)
    return round(score,4)   

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials)

best_params = study.best_params
print("Najlepsze parametry:", best_params)


[I 2025-04-13 22:44:34,014] A new study created in memory with name: no-name-2241ecb4-c471-4a91-9b52-d6bc72d24f3b
[I 2025-04-13 22:45:10,184] Trial 0 finished with value: 0.9772 and parameters: {'n_estimators': 59, 'criterion': 'entropy', 'max_depth': 33, 'min_samples_split': 12, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 0.9772.
[I 2025-04-13 22:46:11,677] Trial 1 finished with value: 0.9787 and parameters: {'n_estimators': 80, 'criterion': 'entropy', 'max_depth': 48, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9787.
[I 2025-04-13 22:48:29,540] Trial 2 finished with value: 0.9777 and parameters: {'n_estimators': 188, 'criterion': 'gini', 'max_depth': 24, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9787.
[I 2025-04-13 22:50:52,282] Trial 3 finished with value: 0.9771 and parameters: {'n_estimators': 194, 'criterion': 'log_loss', 'max_dept

Najlepsze parametry: {'n_estimators': 80, 'criterion': 'entropy', 'max_depth': 48, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 'sqrt'}


In [9]:
best_model = RandomForestClassifier(**best_params, random_state=42)
score=evaluate_model_on_folds(best_model,disp=1)

F1:           0.9787
Accuracy:     0.9679
Precision:    0.9717
Sensitivity:  0.9858
Specificity:  0.9150
TP:           47700
FP:           1389
TN:           14951
FN:           687
         Pred 1    Pred 0
True 1    47700      687
True 0    1389     14951


## SVM

In [10]:
def objective(trial):
    C = trial.suggest_float('C', 1e-5, 1e5,log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
        coef0 = trial.suggest_float ('coef0', -1, 1)
        model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree, coef0=coef0, random_state=42)
    else:
        model = SVC(C=C, kernel=kernel, gamma=gamma, random_state=42)
    
    
    score=evaluate_model_on_folds(model)
    return round(score,4)  

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials)

best_params = study.best_params
print("Najlepsze parametry:", best_params)


[I 2025-04-13 23:14:53,212] A new study created in memory with name: no-name-8cfcef7d-c57d-468b-95d0-8e244fb9eccb
[I 2025-04-13 23:18:29,592] Trial 0 finished with value: 0.8555 and parameters: {'C': 1.2770386725976239e-05, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.8555.
[I 2025-04-13 23:20:11,291] Trial 1 finished with value: 0.9675 and parameters: {'C': 135.884689711956, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 1 with value: 0.9675.
[I 2025-04-13 23:24:06,309] Trial 2 finished with value: 0.9244 and parameters: {'C': 0.011970328441774924, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.9675.
[I 2025-04-13 23:30:55,127] Trial 3 finished with value: 0.7108 and parameters: {'C': 0.04141499356569666, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 1 with value: 0.9675.
[I 2025-04-13 23:31:49,844] Trial 4 finished with value: 0.9628 and parameters: {'C': 0.947261534801931, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 1

Najlepsze parametry: {'C': 17431.85057616861, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3, 'coef0': 0.3860570320218555}


In [11]:
best_model = SVC(**best_params, random_state=42)
score=evaluate_model_on_folds(best_model,disp=1)

F1:           0.9822
Accuracy:     0.9734
Precision:    0.9796
Sensitivity:  0.9849
Specificity:  0.9392
TP:           47656
FP:           993
TN:           15347
FN:           731
         Pred 1    Pred 0
True 1    47656      731
True 0     993     15347
